```
character   : str
attack_name : str
attack_type : primary | secundary | ultimate
damage_tier : higher | lower
damage      : int
```

In [1]:
import re
from itertools import chain, zip_longest
from urllib.request import Request, urlopen

import pandas as pd
from parsel import Selector

In [2]:
_ = 'https://gamefaqs.gamespot.com/boards/939644-dragon-ball-z-budokai-tenkaichi-3/55461581'
_ = Request(
    _,
    headers={
        'User-Agent': (
            'Mozilla/5.0 (X11; Linux x86_64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/68.0.3440.75 Safari/537.36'
        )
    }
)
_ = urlopen(_)
_ = _.read().decode(_.headers.get_charset() or 'utf-8')

html = _
html[:100]

'<!doctype html>\r\n<html lang="en" class="js no-touch" itemscope>\r\n\t<head>\r\n\t\t<title>Blast 2/Ultimate '

In [3]:
_ = Selector(html)
_ = _.css('.msg_body::text')
_ = _.extract()
_ = _[2:]
_ = map(lambda s: s.strip(), _)
_ = filter(len, _)
_ = zip_longest(*[iter(_)]*4)
_ = list(_)

characters = _
characters[:5]

[('Goku(Early)',
  'Kamehameha: 7020-11280',
  'Kaioken Attack: 8000',
  'Spirit Bomb:14010'),
 ('Gohan(Kid)',
  'Masenko:7000-11160',
  'Gekiretsu Madan: 5500',
  'Ultimate: 14760'),
 ('Piccolo (Early)',
  'Bakurikimaha: 7560',
  'Rush Blast 2: 7510',
  'Makankosappo: 15010'),
 ('Vegeta(Scouter)',
  'Galick Hoo: 9520',
  'Rush Blast 2: 7510',
  'Ultimate: 14010'),
 ('Vegeta (Scouter) Great Ape',
  'Mouth Beam Blast 2: 8560',
  'Mouth Beam Barrage Blast 2: 6510',
  'Super Galick Hoo?: 15050')]

In [4]:
def parse_character(character_tuple):
    character, primary, secundary, ultimate = character_tuple

    return [
        {
            'character'   : character,
            'attack_type' : attack_type,
            'attack_name' : attack['attack_name'],
            'damage_tier' : attack['damage_tier'],
            'damage'      : attack['damage'],
        }
        for attack_type, attacks in zip(
            ['primary', 'secundary', 'ultimate'],
            map(parse_attack, [primary, secundary, ultimate]),
        )
        for attack in attacks
    ]


def parse_attack(raw):
    attack_name, damage = re.split(r'\s*:\s*', raw)

    return [
        {
            'attack_name' : attack_name,
            'damage_tier' : damage_tier,
            'damage'      : damage,
        }
        for damage_tier, damage in zip(
            ['lower', 'higher'],
            parse_damage(damage),
        )
    ]


def parse_damage(raw):
    damages = re.findall(r'\d+', raw)
    if len(damages) == 1:
        damages = damages * 2
    lower, higher = map(int, damages)

    return [lower, higher]

In [5]:
_ = characters
_ = map(parse_character, _)
_ = chain.from_iterable(_)
_ = list(_)
_ = pd.DataFrame(_)

df = _
df.head()

,attack_name,attack_type,character,damage,damage_tier
0,Kamehameha,primary,Goku(Early),7020,lower
1,Kamehameha,primary,Goku(Early),11280,higher
2,Kaioken Attack,secundary,Goku(Early),8000,lower
3,Kaioken Attack,secundary,Goku(Early),8000,higher
4,Spirit Bomb,ultimate,Goku(Early),14010,lower


In [6]:
df.to_excel('./attacks.xlsx', index=None)
pd.read_excel('./attacks.xlsx').head()

,attack_name,attack_type,character,damage,damage_tier
0,Kamehameha,primary,Goku(Early),7020,lower
1,Kamehameha,primary,Goku(Early),11280,higher
2,Kaioken Attack,secundary,Goku(Early),8000,lower
3,Kaioken Attack,secundary,Goku(Early),8000,higher
4,Spirit Bomb,ultimate,Goku(Early),14010,lower


In [7]:
(
    df
    .iloc[
        df
        .groupby(['attack_type', 'damage_tier'])
        .damage
        .idxmax()
        .values
    ]
    .set_index(['attack_type', 'damage_tier'])
    .sort_index()
)

attack_name           character  damage
attack_type damage_tier                                                  
primary     higher               Makankosappo        Piccolo(End)   14780
            lower                  Galick Hoo     Vegeta(Scouter)    9520
secundary   higher       Super Explosive Wave        Super Gogeta   10080
            lower        Super Explosive Wave        Super Gogeta   10080
ultimate    higher           Bros. Kamehameha  Gohan Super Saiyan   17000
            lower            Bros. Kamehameha  Gohan Super Saiyan   17000